# Demo and Comparission of Big Data File Formats
This Notebook runs only on a local Spark Environment, not on Kubernetes

## 1. CSV and JSON
Old dat formats that are not designed for big data and scaling  
**Typical feature:** humand readable

## 2. Avro, OCR, Parquet
First generation of special big data formats that allow fast writes, fast reads or both  
**Typical features:** splittable, compressible, data skipping and predicat pushdown, data schema inclueded



## 3. Delta, Iceberg, Hudi
Latest generation of big data format that support ACID transaction, audit save transaction logs and time travel  
**Typical features:** enhancing first generation format with additonal meta data and read/write procedures.

In [5]:
#################################################################################
# Load all relevant Python Modules
#################################################################################
from pyspark import SparkContext, SparkConf
from pyspark.sql import SparkSession
from pyspark.sql import SQLContext
from pyspark.sql.types import *
import pyspark.sql.functions as f

from delta import *

import datetime
from datetime import datetime, timedelta

import json
import csv

# use 95% of the screen for jupyter cell
from IPython.core.display import display, HTML
display(HTML("<style>.container {width:100% !important; }<style>"))

In [2]:
# first for local usage pip install delta-spark

builder = pyspark.sql.SparkSession.builder.appName("MyApp") \
    .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension") \
    .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension") \
    .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog") \
    .config("spark.jars", "/Users/alor/opt/spark/jars/spark-sql-kafka-0-10_2.12-3.3.1.jar, /Users/alor/opt/spark/jars/kafka-clients-3.3.1.jar, /Users/alor/opt/spark/jars/spark-avro_2.12-3.3.1.jar") \
    .config("spark.driver.extraClassPath","/Users/alor/opt/spark/jars/spark-sql-kafka-0-10_2.12-3.3.1.jar, /Users/alor/opt/spark/jars/kafka-clients-3.3.1.jar, /Users/alor/opt/spark/jars/spark-avro_2.12-3.3.1.jar") \
    .config("spark.executor.extraClassPath","/Users/alor/opt/spark/jars/spark-sql-kafka-0-10_2.12-3.3.1.jar, /Users/alor/opt/spark/jars/kafka-clients-3.3.1.jar, /Users/alor/opt/spark/jars/spark-avro_2.12-3.3.1.jar")


spark = configure_spark_with_delta_pip(builder).getOrCreate()

In [3]:
spark

## Create sample data

In [6]:
# initial Daten
account_data1 = [
    (1,"alex","2019-01-01",1000),
    (2,"alex","2019-02-01",1500),
    (3,"alex","2019-03-01",1700),
    (4,"maria","2020-01-01",5000)
    ]

# Datensatz mit einem Update und einer neuen Zeile
account_data2 = [
    (1,"alex","2019-03-01",3300),
    (2,"peter","2021-01-01",100)
    ]

# Datensatz mit neuer Zeile und neuer Spalte
account_data3 = [
    (1,"otto","2019-10-01",4444,"neue Spalte 1")
]

# Datensatz mit neuer Zeile und neuer Spalte
account_data4 = [
    (5,"markus","2019-09-01",555)
]

schema = ["id","account","dt_transaction","balance"]
schema3 = ["id","account","dt_transaction","balance","new"]

df1 = spark.createDataFrame(data=account_data1, schema = schema).withColumn("dt_transaction",f.col("dt_transaction").cast("date")).repartition(3)
df2 = spark.createDataFrame(data=account_data2, schema = schema).withColumn("dt_transaction",f.col("dt_transaction").cast("date")).repartition(2)
df3 = spark.createDataFrame(data=account_data3, schema = schema3).withColumn("dt_transaction",f.col("dt_transaction").cast("date")).repartition(1)
df4 = spark.createDataFrame(data=account_data4, schema = schema).withColumn("dt_transaction",f.col("dt_transaction").cast("date")).withColumn("id",f.col("id").cast("string")).repartition(1)


print("++ create new dataframe and show schema and data")
print("################################################")

# df1.printSchema()
print("++ start data")
df1.show(truncate=False)
print("++ update row and add row")
df2.show(truncate=False)
print("++ add new column")
df3.show(truncate=False)
print("++ add new row with wrong schema (id)")
df4.show(truncate=False)
df1.printSchema()
df4.printSchema()

++ create new dataframe and show schema and data
################################################
++ start data
+---+-------+--------------+-------+
|id |account|dt_transaction|balance|
+---+-------+--------------+-------+
|2  |alex   |2019-02-01    |1500   |
|3  |alex   |2019-03-01    |1700   |
|1  |alex   |2019-01-01    |1000   |
|4  |maria  |2020-01-01    |5000   |
+---+-------+--------------+-------+

++ update row and add row
+---+-------+--------------+-------+
|id |account|dt_transaction|balance|
+---+-------+--------------+-------+
|1  |alex   |2019-03-01    |3300   |
|2  |peter  |2021-01-01    |100    |
+---+-------+--------------+-------+

++ add new column
+---+-------+--------------+-------+-------------+
|id |account|dt_transaction|balance|new          |
+---+-------+--------------+-------+-------------+
|1  |otto   |2019-10-01    |4444   |neue Spalte 1|
+---+-------+--------------+-------+-------------+

++ add new row with wrong schema (id)
+---+-------+--------------+--

## CSV

In [8]:
print("Number of Partitions:", df1.rdd.getNumPartitions())

# Schreibe Datenset 1 als CSV Datei
write_csv=(df1
           .write
           .format("csv")
           .mode("overwrite") # append
           .save("output/csv")
          )


Number of Partitions: 3


In [9]:
!ls output/csv/

_SUCCESS
part-00000-3bb47c50-3e8a-4447-bcc4-bc92f1b50767-c000.csv
part-00001-3bb47c50-3e8a-4447-bcc4-bc92f1b50767-c000.csv
part-00002-3bb47c50-3e8a-4447-bcc4-bc92f1b50767-c000.csv


In [9]:
! cat output/csv/part-00002-1969e3ad-db5e-4e0a-8214-4a9869c24d11-c000.csv

4,maria,2020-01-01,5000


In [10]:
read_csv=spark.read.format("csv").load("output/csv")

read_csv.printSchema()
read_csv.show()

root
 |-- _c0: string (nullable = true)
 |-- _c1: string (nullable = true)
 |-- _c2: string (nullable = true)
 |-- _c3: string (nullable = true)

+---+-----+----------+----+
|_c0|  _c1|       _c2| _c3|
+---+-----+----------+----+
|  2| alex|2019-02-01|1500|
|  3| alex|2019-03-01|1700|
|  4|maria|2020-01-01|5000|
|  1| alex|2019-01-01|1000|
+---+-----+----------+----+



In [11]:
# schreibe Datenset 3 (neue Spalte) in die gleiche Tabelle dazu
write_csv=(df3
           .write
           .format("csv")
           .mode("append") # append
           .save("output/csv")
          )

In [12]:
!ls output/csv/

_SUCCESS
part-00000-1969e3ad-db5e-4e0a-8214-4a9869c24d11-c000.csv
part-00000-411ddc45-144a-4a3d-900f-90a6bf3c46bf-c000.csv
part-00001-1969e3ad-db5e-4e0a-8214-4a9869c24d11-c000.csv
part-00002-1969e3ad-db5e-4e0a-8214-4a9869c24d11-c000.csv
part-00002-411ddc45-144a-4a3d-900f-90a6bf3c46bf-c000.csv


In [16]:
!cat output/csv/part-00000-411ddc45-144a-4a3d-900f-90a6bf3c46bf-c000.csv

In [17]:
# und lese alles nochmal ein um zu schauen ob die neue Spalte richtig erkannt wurde
read_csv=spark.read.format("csv").load("output/csv")

read_csv.printSchema()
read_csv.show()

root
 |-- _c0: string (nullable = true)
 |-- _c1: string (nullable = true)
 |-- _c2: string (nullable = true)
 |-- _c3: string (nullable = true)

+---+-----+----------+----+
|_c0|  _c1|       _c2| _c3|
+---+-----+----------+----+
|  2| alex|2019-02-01|1500|
|  3| alex|2019-03-01|1700|
|  1| otto|2019-10-01|4444|
|  4|maria|2020-01-01|5000|
|  1| alex|2019-01-01|1000|
+---+-----+----------+----+



#### Erkenntnisse CSV
* In wieviele Dateien wird das Datenset aufgeteilt und warum?
* Bleibt das Schema erhalten (Selbsterklärend)
* Können neue Spalten angefügt werden (Schema Evolution)

## JSON

In [11]:
print("Number of Partitions:", df1.rdd.getNumPartitions())

write_json=(df1
           .write
           .format("json")
           .mode("overwrite") # append
           .save("output/json")
          )


Number of Partitions: 3


In [12]:
!ls output/json/

_SUCCESS
part-00000-e4a7d280-0fa8-484b-abee-0feabe89857a-c000.json
part-00001-e4a7d280-0fa8-484b-abee-0feabe89857a-c000.json
part-00002-e4a7d280-0fa8-484b-abee-0feabe89857a-c000.json


In [20]:
! cat output/json/part-00000-8f1197ca-4955-4b8a-a050-010b3eba7e2e-c000.json

{"id":2,"account":"alex","dt_transaction":"2019-02-01","balance":1500}
{"id":3,"account":"alex","dt_transaction":"2019-03-01","balance":1700}


In [52]:
write_csv=(df3
           .write
           .format("json")
           .mode("append") # append
           .save("output/json")
          )

In [55]:
read_json=spark.read.format("json").load("output/json")

read_json.printSchema()
read_json.show()

root
 |-- account: string (nullable = true)
 |-- balance: string (nullable = true)
 |-- dt_transaction: string (nullable = true)
 |-- id: long (nullable = true)
 |-- new: string (nullable = true)

+-------+-------+--------------+---+-------------+
|account|balance|dt_transaction| id|          new|
+-------+-------+--------------+---+-------------+
|   alex|   1500|    2019-02-01|  2|         null|
|   alex|   1700|    2019-03-01|  3|         null|
|   otto|   4444|    2019-10-01|  1|neue Spalte 1|
|   otto|   4444|    2019-10-01|  1|neue Spalte 1|
|  maria|   5000|    2020-01-01|  4|         null|
|   alex|   1000|    2019-01-01|  1|         null|
+-------+-------+--------------+---+-------------+



#### Erkenntnisse JSON
* Bleibt das Schema erhalten (Selbsterklärend)?
* Können neue Spalten angefügt werden (Schema Evolution)?
* Gibt es Schema Enforcement?

## Avro

In [15]:
print("Number of Partitions:", df1.rdd.getNumPartitions())
# Schreibe Datenset 1 als AVRO Datei
write_avro=(df1
           .write
           .format("avro")
           .mode("overwrite") # append
           .save("output/avro")
          )

Number of Partitions: 3


In [16]:
!ls output/avro/

_SUCCESS
part-00000-1fcb830b-c287-4fea-b4f0-b23504fb105a-c000.avro
part-00001-1fcb830b-c287-4fea-b4f0-b23504fb105a-c000.avro
part-00002-1fcb830b-c287-4fea-b4f0-b23504fb105a-c000.avro


In [25]:
! cat output/avro/part-00000-f47ea23a-bd11-4456-9c6e-dd455ffeecb3-c000.avro

Objavro.schema�{"type":"record","name":"topLevelRecord","fields":[{"name":"id","type":["long","null"]},{"name":"account","type":["string","null"]},{"name":"dt_transaction","type":[{"type":"int","logicalType":"date"},"null"]},{"name":"balance","type":["long","null"]}]}0org.apache.spark.version
3.1.2avro.codecsnappy ����!ɸ �9
Ht  alex �� �  alex Ș �3�������!ɸ �9


In [17]:
read_json=spark.read.format("avro").load("output/avro")
read_json.printSchema()

root
 |-- id: long (nullable = true)
 |-- account: string (nullable = true)
 |-- dt_transaction: date (nullable = true)
 |-- balance: long (nullable = true)



In [18]:
# schreibe Datenset 3 (neue Spalte) in die gleiche Tabelle dazu (!! append NOT overwrite)
write_avro=(df3
           .write
           .format("avro")
           .mode("append") # append
           .save("output/avro")
          )

In [28]:
read_json=spark.read.format("avro").load("output/avro")
read_json.printSchema()
read_json.show()

root
 |-- id: long (nullable = true)
 |-- account: string (nullable = true)
 |-- dt_transaction: date (nullable = true)
 |-- balance: long (nullable = true)

+---+-------+--------------+-------+
| id|account|dt_transaction|balance|
+---+-------+--------------+-------+
|  1|   otto|    2019-10-01|   4444|
|  2|   alex|    2019-02-01|   1500|
|  3|   alex|    2019-03-01|   1700|
|  4|  maria|    2020-01-01|   5000|
|  1|   alex|    2019-01-01|   1000|
+---+-------+--------------+-------+



* Schema erhalten
* Schema evolution 

## Parquet

In [19]:
print("Number of Partitions:", df1.rdd.getNumPartitions())

write_parquet=(df1
           .write
           # Fachliche Partitionierung beim Schreiben
           .partitionBy("account")
           .format("parquet")
           .mode("overwrite") # append
           .save("output/parquet")
          )


Number of Partitions: 3


In [24]:
df1.show()

+---+-------+--------------+-------+
| id|account|dt_transaction|balance|
+---+-------+--------------+-------+
|  2|   alex|    2019-02-01|   1500|
|  3|   alex|    2019-03-01|   1700|
|  1|   alex|    2019-01-01|   1000|
|  4|  maria|    2020-01-01|   5000|
+---+-------+--------------+-------+



In [33]:
!ls  output/parquet/
!echo "----------------------------"
!ls output/parquet/account=alex

_SUCCESS      account=alex  account=maria
----------------------------
part-00000-aacb8d79-4737-47c1-97a9-54c1dec538c3.c000.snappy.parquet
part-00001-aacb8d79-4737-47c1-97a9-54c1dec538c3.c000.snappy.parquet


In [34]:
! cat output/parquet/account=alex/part-00000-aacb8d79-4737-47c1-97a9-54c1dec538c3.c000.snappy.parquet

PAR1 ,.,                (                    ,             , $F  F   ($F  F     4   F  $F   ,0, �      �       (�      �         T   �      �      LHspark_schema %id %dt_transaction% %balance <&5 id��&<               (                    &�5 dt_transactionvz&�<$F  F   ($F  F        &�5 balance��&�<�      �       (�      �            � ,org.apache.spark.version3.1.2 )org.apache.spark.sql.parquet.row.metadata�{"type":"struct","fields":[{"name":"id","type":"long","nullable":true,"metadata":{}},{"name":"dt_transaction","type":"date","nullable":true,"metadata":{}},{"name":"balance","type":"long","nullable":true,"metadata":{}}]} Jparquet-mr version 1.10.1 (build a89df8f9932b6ef6633d06069e50c9b7970bebd1)<       �  PAR1

In [35]:
read_parquet=spark.read.format("parquet").load("output/parquet/").filter(col("account")=="alex").show()

+---+--------------+-------+-------+
| id|dt_transaction|balance|account|
+---+--------------+-------+-------+
|  2|    2019-02-01|   1500|   alex|
|  3|    2019-03-01|   1700|   alex|
|  1|    2019-01-01|   1000|   alex|
+---+--------------+-------+-------+



### Parquet: Filter Pushdown
Da das Parquet Format spalten basiert ist und für jede Spalte Metadaten vorhällt, können Programme die diese Dateien einlesen vor der Serialisierung (dem kompletten Einlesen in den Arbeitsspeicher) erst die Header scannen und entscheiden welche Dateien tatsächlich benötigt werden.  
Dies nennt man Attribut oder Filter Pushdown.  
Spark kann außerdem, wenn die Daten in Partition im Format `PartitionKey=value` diese automatisch erkennen und wenn ein Filter auf die Partition gelegt ist nur diesen Unterordner einlesen.  

**Aufgabe:** Untersuche den Execution Plan der Filter Operation 

In [22]:
# Parquet Datei mit PartitionFilter laden
read_parquet=(spark
              .read.format("parquet")
              .load(f"output/parquet")
              # Filter auf die Spalte über die partitioniert wurde
              .filter(f.col("account")=="alex")
             )

# Parquet mit normalem Filter laden
read_parquet2=(spark
              .read.format("parquet")
              .load(f"output//parquet")
              # Filter auf die Spalte über eine normale Spalte
              .filter(f.col("balance")>1500)
             )

# Anzeigen des physischen Execution Plans um zu sehen welche Filter ins Dateisystem bzw. in die Parquet Datei gepusht werden
print("Partition Filter")
read_parquet.explain("simple")
print("Pushdow Filter")
read_parquet2.explain("simple")


Partition Filter
== Physical Plan ==
*(1) ColumnarToRow
+- FileScan parquet [id#787L,dt_transaction#788,balance#789L,account#790] Batched: true, DataFilters: [], Format: Parquet, Location: InMemoryFileIndex[file:/Users/alor/All/git/big-data-on-k8s-workshop/4_demos/output/parquet], PartitionFilters: [isnotnull(account#790), (account#790 = alex)], PushedFilters: [], ReadSchema: struct<id:bigint,dt_transaction:date,balance:bigint>


Pushdow Filter
== Physical Plan ==
*(1) Filter (isnotnull(balance#797L) AND (balance#797L > 1500))
+- *(1) ColumnarToRow
   +- FileScan parquet [id#795L,dt_transaction#796,balance#797L,account#798] Batched: true, DataFilters: [isnotnull(balance#797L), (balance#797L > 1500)], Format: Parquet, Location: InMemoryFileIndex[file:/Users/alor/All/git/big-data-on-k8s-workshop/4_demos/output/parquet], PartitionFilters: [], PushedFilters: [IsNotNull(balance), GreaterThan(balance,1500)], ReadSchema: struct<id:bigint,dt_transaction:date,balance:bigint>




### Parquet: Schema Evolution
Schema Evolution ermöglicht es das Schema der Tabelle zu erweitern.  
Der Spark Parquet reader bietet verschiedenen Möglichkeiten mit Schemaerweiterungen umzugehen  
https://spark.apache.org/docs/latest/sql-data-sources-parquet.html#schema-merging

**Aufgabe:** Lese die Daten so ein, dass das Schema korrekt erweitert wird

In [23]:
# Zeile mit neuer Spalte anfügen
write_parquet=(df3
           .write
           .format("parquet")
           .mode("append") # append
           # schreibe ohne zu Partitionieren direkt in ein neues Unterverzeichnis
           .save(f"output/parquet/account=otto")
          )

In [24]:
# einlesen mit der mergeSchema Option
read_parquet=(spark
              .read.format("parquet")
              # setzte die mergeSchema auf true/false um den Unterschied beim Einlesen zu sehen
              # Vegleiche: https://spark.apache.org/docs/latest/sql-data-sources-parquet.html#schema-merging
              #.option("mergeSchema", "false")
              .load(f"output/parquet")
             )

read_parquet.printSchema()
read_parquet.show()

root
 |-- id: long (nullable = true)
 |-- dt_transaction: date (nullable = true)
 |-- balance: long (nullable = true)
 |-- account: string (nullable = true)

+---+--------------+-------+-------+
| id|dt_transaction|balance|account|
+---+--------------+-------+-------+
|  1|    2019-10-01|   4444|   otto|
|  2|    2019-02-01|   1500|   alex|
|  3|    2019-03-01|   1700|   alex|
|  4|    2020-01-01|   5000|  maria|
|  1|    2019-01-01|   1000|   alex|
+---+--------------+-------+-------+



### Parquet: Schema Enforcement
Schema Enforcement sorgt dafür, dass in ein bestehendes Schema keine Daten mit falschen Typen gelesen oder geschrieben werden können

In [26]:
# Datensatz mit falschem Datentyp anfügen
df2a=(df2.where(f.col("account")=="peter").withColumn("id", f.col("id").cast("string")))


# Zeile mit falschem Typ anfügen
write_parquet=(df2a
           .write
           .partitionBy("account")
           .format("parquet")
           .mode("append") # append
           .save(f"output/parquet")
          )

In [27]:
# einlesen mit der mergeSchema Option
read_parquet=(spark
              .read.format("parquet")
              # setzte die mergeSchema auf true/false um den Unterschied beim Einlesen zu sehen
              .option("mergeSchema", "false")
              .load(f"output/parquet")
             )

read_parquet.printSchema()
read_parquet.show()

root
 |-- id: long (nullable = true)
 |-- dt_transaction: date (nullable = true)
 |-- balance: long (nullable = true)
 |-- account: string (nullable = true)



Py4JJavaError: An error occurred while calling o237.showString.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 3 in stage 48.0 failed 1 times, most recent failure: Lost task 3.0 in stage 48.0 (TID 217) (macbook-thinkport.fritz.box executor driver): org.apache.spark.sql.execution.QueryExecutionException: Parquet column cannot be converted in file file:///Users/alor/All/git/big-data-on-k8s-workshop/4_demos/output/parquet/account=peter/part-00000-a447ca37-2532-4021-8346-4040597cbc23.c000.snappy.parquet. Column: [id], Expected: bigint, Found: BINARY
	at org.apache.spark.sql.execution.datasources.FileScanRDD$$anon$1.nextIterator(FileScanRDD.scala:179)
	at org.apache.spark.sql.execution.datasources.FileScanRDD$$anon$1.hasNext(FileScanRDD.scala:93)
	at org.apache.spark.sql.execution.FileSourceScanExec$$anon$1.hasNext(DataSourceScanExec.scala:503)
	at org.apache.spark.sql.catalyst.expressions.GeneratedClass$GeneratedIteratorForCodegenStage1.columnartorow_nextBatch_0$

#### Erkenntnisse Parquet
* Sind Parquet Dateien selbsterklärend (haben ein Spalten und Typenschema )
* Partitioning and Partion Discovery: werden die Daten in Verzeichnisse geschriebe und wieder als Partitionen erkannt?
* Schema Evolution: Kann das Schema erweitert werden, also eine neue Spalte angefügt werden?
* Schema Enforcement on write: Kann eine Spalte mit falschem Datetyp einfach beim schreiben hinzugefügt werden? 
* Schema Enforcement on read: Kann ein Verzeichnis mit mehreren Parquet Dateien bei der eine Spalte ein anderes Schema hat gelesen werden?

# Delta
Das Delta Format fügt Parquet Dateien einen zusätzlichen Layer an Metadatan hinzu und erfüllt mit dem entsprechenden Treiber alle ACID Eigenschaften einer Datenbank und mehr. 

A = **Atomic** heißt alle Datenänderungen werden wie eine einzige Operation verarbeitet. Dies bedeutet, dass entweder alle Änderungen durchgeführt werden oder keine. Wenn das schreiben also mitten drin Fehlschlägt werden alle Daten dieser Schreiboperation die bereits geschrieben wurden wieder entfernt, bzw. nicht als erfolgreich geschrieben markiert.  
T = **Consistency** bedeutet, wenn eine Transaktion beginnt und wenn eine Transaktion endet, befinden sich die Daten in einem konsistenten Zustand.  
I = **Isolation** und bedeutet, dass der Übergangszustand einer Transaktion für andere Transaktionen nicht sichtbar ist. Dies führt dazu, dass Transaktionen, die gleichzeitig ablaufen, sich nicht gegenseitig beeinflussen oder blockieren.  
D =**Durability** heißt, das die Datenänderungen nach erfolgreich abgeschlossener Transaktion erhalten bleiben und werden nicht rückgängig gemacht werden, selbst wenn ein Systemausfall auftritt.  

**Time Travel** bei Delta bedeutet, dass jede Datenänderung als eigene Version aufgezeichnet wird und jederzeit zu einer alten Version zurück gegegangen werden kann.

### Aufgabe:
Wiederhole die gleichen Schritte mit dem DELTA Format und schaue wie sich hier Schema und neue Spalten verhalten

1. Datenset 1 als DELTA schreiben (.format("delta") und Pfad= .save(f"s3://{bucket}/delta"))
2. Dateien und Inhalt anzeigen, vestehen was da passiert ist
3. Metadaten und Deltalog in Datei verstehen
3. Daten wieder einlese und checken ob es ein Schema und Spaltennamen gibt
4. Schema Evolutiuon: Datenset 3 anfügen mit neuer Spalte anfügen
5. Daten wieder einlesen und checken was mit der neuen Spalte passiert
6. Partion & Pushdown Filter: Execution Plan für verschiedene Filter anzeigen
6. Schema Enforcement: Datentyp in bestehender Spalte ändern und schauen ob und wie dies gehandhabt wird

In [28]:
# Schreibe die Daten df1 als Delta Datei in den Pfad bucket/delta
write_delta=(df1
           .write
           .format("delta")
           #.option("mergeSchema", "false")
           .mode("overwrite") 
           .save(f"output/delta")
          )

In [30]:
! ls output/delta
! echo "-------------------------------------------------------------------"
! ls output/delta/_delta_log
! cat output/delta/_delta_log/00000000000000000000.json

_delta_log
part-00000-c5db6f76-a542-45fe-bcf4-e3e7ef46e9d3-c000.snappy.parquet
part-00000-dada16c9-a473-4ded-a0c1-deb66d743bc0-c000.snappy.parquet
part-00001-4e386600-5d7f-45a6-9152-5c8ef563d551-c000.snappy.parquet
part-00001-96342876-21c2-450c-9a4e-f8f4223b4f72-c000.snappy.parquet
part-00002-b32284bf-bd8b-4c80-85b7-037aa0802901-c000.snappy.parquet
part-00002-cba791ad-9a06-493d-bc03-a565dccc6c0b-c000.snappy.parquet
-------------------------------------------------------------------
00000000000000000000.json 00000000000000000001.json
{"commitInfo":{"timestamp":1689839781730,"operation":"WRITE","operationParameters":{"mode":"Overwrite","partitionBy":"[]"},"isBlindAppend":false,"operationMetrics":{"numFiles":"3","numOutputBytes":"3527","numOutputRows":"4"}}}
{"protocol":{"minReaderVersion":1,"minWriterVersion":2}}
{"metaData":{"id":"1da24135-4733-4b1e-b5b3-9ebbc720d55d","format":{"provider":"parquet","options":{}},"schemaString":"{\"type\":\"struct\",\"fields\":[{\"name\":\"id\",\"type\":

In [32]:
# lese die gerade geschriebenen Delta Tabelle wieder ein und zeige sie an, überprüfe das Schema
read_delta=(spark
              .read.format("delta")
              .load(f"output/delta")
             )
read_delta.printSchema()
read_delta.show()

root
 |-- id: long (nullable = true)
 |-- account: string (nullable = true)
 |-- dt_transaction: date (nullable = true)
 |-- balance: long (nullable = true)

+---+-------+--------------+-------+
| id|account|dt_transaction|balance|
+---+-------+--------------+-------+
|  2|   alex|    2019-02-01|   1500|
|  3|   alex|    2019-03-01|   1700|
|  4|  maria|    2020-01-01|   5000|
|  1|   alex|    2019-01-01|   1000|
+---+-------+--------------+-------+



### Delta: Schema Evolution
**Aufgabe:** Verstehe die Option *mergeSchema* on write

In [33]:
# Zeile mit zusätzlicher Spalte anfügen (df3)
write_delta=(df3
           .write
           .format("delta")
           # Bei Delta kann bein Schreiben gesetzt werden ob die Tabelle erweitert werden soll oder nicht, Default ist false. 
           # Führe den Code zuerst ohne diese Option aus und schaue das Ergebnis, an 
           #.option("mergeSchema", "false")
           .mode("append") # append
           .save(f"output/delta")
          )


# überprüfe ob die neue Spalte korrekt angefügt wurde
read_delta=spark.read.format("delta").load(f"output/delta")
read_delta.show()

AnalysisException: A schema mismatch detected when writing to the Delta table (Table ID: 1da24135-4733-4b1e-b5b3-9ebbc720d55d).
To enable schema migration using DataFrameWriter or DataStreamWriter, please set:
'.option("mergeSchema", "true")'.
For other operations, set the session configuration
spark.databricks.delta.schema.autoMerge.enabled to "true". See the documentation
specific to the operation for details.

Table schema:
root
-- id: long (nullable = true)
-- account: string (nullable = true)
-- dt_transaction: date (nullable = true)
-- balance: long (nullable = true)


Data schema:
root
-- id: long (nullable = true)
-- account: string (nullable = true)
-- dt_transaction: date (nullable = true)
-- balance: long (nullable = true)
-- new: string (nullable = true)

         

In [34]:
! ls output/delta
! echo "-------------------------------------------------------------------"
! ls output/delta/_delta_log

_delta_log
part-00000-c5db6f76-a542-45fe-bcf4-e3e7ef46e9d3-c000.snappy.parquet
part-00000-dada16c9-a473-4ded-a0c1-deb66d743bc0-c000.snappy.parquet
part-00001-4e386600-5d7f-45a6-9152-5c8ef563d551-c000.snappy.parquet
part-00001-96342876-21c2-450c-9a4e-f8f4223b4f72-c000.snappy.parquet
part-00002-b32284bf-bd8b-4c80-85b7-037aa0802901-c000.snappy.parquet
part-00002-cba791ad-9a06-493d-bc03-a565dccc6c0b-c000.snappy.parquet
-------------------------------------------------------------------
00000000000000000000.json 00000000000000000001.json


In [ ]:
! cat output/delta/_delta_log/00000000000000000002.json

### Delta: Schema Enforcement
Schema Enforcement bedeutet soll garantieren, dass keine Daten mit falschen Datentyp der Tabelle abgefügt werden  

**Aufgabe:** verstehe die Option *mergeSchema* im Kontext von Datentyp Änderungen bei bestehenden Spalten

In [ ]:
# füge eine Zeile mit falschen Datetyp für eine bestehenden Spalte an
write_delta=(df2
           # Eine Zeile aus dem df2 filtern
           .where(f.col("account")=="peter")
           # Bestehenden Spaltentyp ändern
           .withColumn("id", f.col("id").cast("string"))
           .write
           .format("delta")
           # Bei Delta kann bein Schreiben gesetzt werden ob die Tabelle erweitert werden kann oder nicht, Default ist false
           #.option("mergeSchema", "false")
           .mode("overwrite") # append
           .save(f"s3://{bucket}/delta")
          )

### Delta: Schema Replacement
Delta bieten die Möglichkeit das Schema einer Tabelle zu ändern, also z.B. eine bestehende Spalte umzubenennen und deren Datentyp zu ändern.   
Dokumentation mit Beispielen: https://docs.delta.io/latest/delta-batch.html#replace-table-schema  

**Aufgabe:** Ändere in der bestehenden Tabelle den Datentyp der Spalte `id` auf `string` und den Spaltennamen `new` zu `comment`. 

In [ ]:
read_delta=(spark
              .read.format("delta")
              .load(f"output/delta")
             )

In [ ]:
write_delta=()

### Delta: History und Metadaten
Im Delta Log werde alle Transaktionen mit zahlreichen Metadaten gespeichert. Das Spark Modul, der Treiber, um diese Daten auszulesen bietet zahlreiche Möglichkeiten diese Daten zu analysieren

**Aufgabe:** Lese den History Log ein und verstehe was in den einzelnen Attributen steht. Treffe eine Auswahl der interessanten Informationen

In [36]:
# Erzeuge ein DeltaTable Objekt was alle Zusatzeigenschaften von Delta bereitstellt
deltaTable = DeltaTable.forPath(spark, f"output/delta")

# Historie aus den Delta Logs erzeugen
fullHistoryDF = deltaTable.history() 

# Alle verfügbaren Spalten anzeigen
fullHistoryDF.printSchema()

root
 |-- version: long (nullable = true)
 |-- timestamp: timestamp (nullable = true)
 |-- userId: string (nullable = true)
 |-- userName: string (nullable = true)
 |-- operation: string (nullable = true)
 |-- operationParameters: map (nullable = true)
 |    |-- key: string
 |    |-- value: string (valueContainsNull = true)
 |-- job: struct (nullable = true)
 |    |-- jobId: string (nullable = true)
 |    |-- jobName: string (nullable = true)
 |    |-- runId: string (nullable = true)
 |    |-- jobOwnerId: string (nullable = true)
 |    |-- triggerType: string (nullable = true)
 |-- notebook: struct (nullable = true)
 |    |-- notebookId: string (nullable = true)
 |-- clusterId: string (nullable = true)
 |-- readVersion: long (nullable = true)
 |-- isolationLevel: string (nullable = true)
 |-- isBlindAppend: boolean (nullable = true)
 |-- operationMetrics: map (nullable = true)
 |    |-- key: string
 |    |-- value: string (valueContainsNull = true)
 |-- userMetadata: string (nullable =

In [44]:
fullHistoryDF.select("version","readVersion","timestamp","userId","operation","operationParameters","operationMetrics","userMetadata").show(truncate=True)

+-------+-----------+-------------------+------+---------+--------------------+--------------------+------------+
|version|readVersion|          timestamp|userId|operation| operationParameters|    operationMetrics|userMetadata|
+-------+-----------+-------------------+------+---------+--------------------+--------------------+------------+
|      5|          4|2023-04-11 11:39:56|  null|    WRITE|{mode -> Overwrit...|{numFiles -> 2, n...|        null|
|      4|          3|2023-04-11 11:39:39|  null|    WRITE|{mode -> Append, ...|{numFiles -> 2, n...|        null|
|      3|          2|2023-04-11 11:34:00|  null|    WRITE|{mode -> Overwrit...|{numFiles -> 3, n...|        null|
|      2|          1|2023-03-27 10:23:49|  null|    WRITE|{mode -> Overwrit...|{numFiles -> 2, n...|        null|
|      1|          0|2023-03-27 10:23:34|  null|    WRITE|{mode -> Append, ...|{numFiles -> 2, n...|        null|
|      0|       null|2023-03-27 10:22:22|  null|    WRITE|{mode -> Overwrit...|{numFiles

In [38]:
fullHistoryDF.select("version","readVersion","timestamp","operation","operationParameters.mode","operationMetrics.numFiles","operationMetrics.numOutputRows").show(truncate=False)

+-------+-----------+-------------------+---------+---------+--------+-------------+
|version|readVersion|timestamp          |operation|mode     |numFiles|numOutputRows|
+-------+-----------+-------------------+---------+---------+--------+-------------+
|1      |0          |2023-07-20 10:06:23|WRITE    |Overwrite|3       |4            |
|0      |null       |2023-07-20 09:56:22|WRITE    |Overwrite|3       |4            |
+-------+-----------+-------------------+---------+---------+--------+-------------+



## Time travel
Die Time Travel Funktion von Delta ermöglicht es den Zustand der Datentabelle zu einem bestimmten Zeitpunkt in der Vergangenheit wiederherzustellen oder Änderungen zu verfolgen.   
Time Travel ermöglicht es, vorherige Versionen der Daten abzufragen und historische Analysen durchzuführen, ohne auf separate Backups oder Snapshots angewiesen zu sein.

**Aufgabe:** lese verschiedenen Datenstände nach Versionsnummer oder Timestap ein.  
Weitere Informationen hierzu finden sich in https://delta.io/blog/2023-02-01-delta-lake-time-travel/

In [48]:
spark.read.format("delta").option("versionAsOf", "4").load("output/delta").show()


+---+-------+--------------+-------+----+
| id|account|dt_transaction|balance| new|
+---+-------+--------------+-------+----+
|  2|   alex|    2019-02-01|   1500|null|
|  3|   alex|    2019-03-01|   1700|null|
|  2|  peter|    2021-01-01|    100|null|
|  4|  maria|    2020-01-01|   5000|null|
|  1|   alex|    2019-03-01|   3300|null|
|  1|   alex|    2019-01-01|   1000|null|
+---+-------+--------------+-------+----+



## Merge
### Delta: Merge (Upsert)
Delta bietet die Möglichkeiten auf bestehenden Dateien ein Daten Upsert durchzuführen.   
Ubsert oder Merge bedeutet zu prüfen ob es die neue Zeile für einen bestimmten Schlüssel schon in den Daten gibt und wenn ja diese zu updaten und wenn nein sie neu hinzuzufügen

**Aufgabe:** Merge den Datensatz df2 mit dem der Änderung der `balance` für `Alex` und überprüfe ob das Update korrekt war

In [45]:
deltaTable2 = DeltaTable.forPath(spark, f"output/delta")

# Spalte anfügen, da merge nur funktioniert wenn das Schema stimmt
#df2a=df2.withColumn("new",f.lit("test"))

print("++ Datensatz der auf bestehende Daten upserted/merged werden soll")
df2a.show()
print("++ Bestehender Datensatz in Delta")
deltaTable2.toDF().show()

++ Datensatz der auf bestehende Daten upserted/merged werden soll
+---+-------+--------------+-------+----+
| id|account|dt_transaction|balance| new|
+---+-------+--------------+-------+----+
|  1|   alex|    2019-03-01|   3300|test|
|  2|  peter|    2021-01-01|    100|test|
+---+-------+--------------+-------+----+

++ Bestehender Datensatz in Delta
+---+-------+--------------+-------+
| id|account|dt_transaction|balance|
+---+-------+--------------+-------+
|  4|  maria|    2020-01-01|   5000|
|  2|  peter|    2021-01-01|    100|
|  1|   alex|    2019-01-01|   1000|
|  2|   alex|    2019-02-01|   1500|
|  1|   alex|    2019-03-01|   3300|
+---+-------+--------------+-------+



In [46]:
# Verwendung der merge Funktion (es gibt auch eine update() oder delete() Funktion)
dt3=(deltaTable2.alias("oldData")
      .merge(df2a.alias("newData"),
            "oldData.account = newData.account AND oldData.dt_transaction = newData.dt_transaction")
            .whenMatchedUpdateAll()
            .whenNotMatchedInsertAll()
      .execute()
    )

deltaTable2.toDF().show()

+---+-------+--------------+-------+
| id|account|dt_transaction|balance|
+---+-------+--------------+-------+
|  4|  maria|    2020-01-01|   5000|
|  2|  peter|    2021-01-01|    100|
|  1|   alex|    2019-01-01|   1000|
|  1|   alex|    2019-03-01|   3300|
|  2|   alex|    2019-02-01|   1500|
+---+-------+--------------+-------+



### Delta: Roleback
Delta bietet die Möglichkeiten direkt auf den Datenstand einer bestimmten Version oder eines Zeitpunktes zurück zu gehen

In [50]:
deltaTable2.toDF().show()
deltaTable2.history().select("version","readVersion","timestamp","operation","operationParameters.mode","operationMetrics.numFiles","operationMetrics.numOutputRows").show(truncate=False)
deltaTable2.restoreToVersion(0)
#restoreToTimestamp
#isDeltaTable
deltaTable2.toDF().show()
#spark.read.format("delta").load(f"s3://{bucket}/delta").show()

+---+-------+--------------+-------+
| id|account|dt_transaction|balance|
+---+-------+--------------+-------+
|  4|  maria|    2020-01-01|   5000|
|  2|  peter|    2021-01-01|    100|
|  1|   alex|    2019-01-01|   1000|
|  1|   alex|    2019-03-01|   3300|
|  2|   alex|    2019-02-01|   1500|
+---+-------+--------------+-------+

+-------+-----------+-------------------+---------+---------+--------+-------------+
|version|readVersion|timestamp          |operation|mode     |numFiles|numOutputRows|
+-------+-----------+-------------------+---------+---------+--------+-------------+
|3      |2          |2023-07-20 10:19:18|MERGE    |null     |null    |2            |
|2      |1          |2023-07-20 10:18:12|MERGE    |null     |null    |3            |
|1      |0          |2023-07-20 10:06:23|WRITE    |Overwrite|3       |4            |
|0      |null       |2023-07-20 09:56:22|WRITE    |Overwrite|3       |4            |
+-------+-----------+-------------------+---------+---------+--------+-

AttributeError: 'DeltaTable' object has no attribute 'restoreToVersion'

#### Erkenntnisse Delta
* Wie funktioniert das Metadatenmanagement und was steht im Delta Log?
* Schema Evolution: Kann das Schema erweitert werden, also eine neue Spalte angefügt werden?
* Schema Enforcement on write: Kann eine Spalte mit falschem Datetyp einfach beim schreiben hinzugefügt werden? 
* Schema Enforcement on read: Kann ein Verzeichnis mit mehreren Parquet Dateien bei der eine Spalte ein anderes Schema hat gelesen werden? (um die Ecke Denk Frage)
* Was ermöglichen mir die Metadaten (Historie, Audit etc)
* Was ist der Vorteil der Merge Funktion? Wie müsste ich sonst Dateibasiert einen Merge/Update durchführen?


In [48]:
spark.stop()

In [51]:
! pip list

Package                       Version
----------------------------- -------------------
aiobotocore                   2.3.4
aiohttp                       3.8.1
aioitertools                  0.10.0
aiosignal                     1.2.0
appnope                       0.1.2
argon2-cffi                   20.1.0
async-generator               1.10
async-timeout                 4.0.2
attrs                         21.2.0
backcall                      0.2.0
bleach                        3.3.1
boto3                         1.22.2
botocore                      1.24.21
Bottleneck                    1.3.2
cachetools                    5.2.0
certifi                       2021.10.8
cffi                          1.14.6
charset-normalizer            2.0.12
click                         8.1.3
cryptography                  37.0.2
decorator                     5.0.9
defusedxml                    0.7.1
delta                         0.4.2
delta-spark                   2.1.1
entrypoints                   0.3
fr